# 异步爬虫
    - 基于线程池的异步爬虫
    - 基于多任务异步协程（重点）


## 线程池

In [2]:
from multiprocessing.dummy import Pool #创建线程池
import time
import requests

## 同步的代码
urls=[
    'http://127.0.0.1:5000/sxl',
    'http://127.0.0.1:5000/bobo',
    'http://127.0.0.1:5000/tom',
]

def get_request(url):
    page_text=requests.get(url).text
    print(len(page_text))

if __name__=="__main__":
    start=time.time()
    for url in urls:
        get_request(url)
    print('总耗时：',time.time()-start)

1271
1271
1271
总耗时： -6.035472869873047


In [5]:
## 异步代码
from multiprocessing.dummy import Pool #创建线程池
import time
import requests

## 同步的代码
urls=[
    'http://127.0.0.1:5000/sxl',
    'http://127.0.0.1:5000/bobo',
    'http://127.0.0.1:5000/tom',
]

def get_request(url):
    page_text=requests.get(url).text
    return (len(page_text))

if __name__=="__main__":
    start=time.time()
    pool=Pool(3)
    page_text_list=pool.map(get_request,urls)
    print(page_text_list)
    ##参数1：回调函数
    ## 参数2：可迭代的对象，alist
    #作用：将alist中每一个元素一次传递回调函数作为参数，让后回调函数会异步对列表中的元素进行相关的操作
    #map的返回值是回调函数返回的所有结果
    print('总耗时：',time.time()-start)

[1271, 1271, 1271]
总耗时： 4.0352561473846436


## 多任务异步协程[asyncio]
    - 环境安装
     - pip insatll asyncio
    - 特殊函数
     - 如果一个函数的定义被async关键字修饰，该函数就变成一个特殊函数
     - 特殊之处
        - 该函数调用之后(函数内部的实现语句)不会被[立即]执行
        - 该函数被调用之后会返回一个协程对象
    - 协程
        - 对象，该对象是特殊的函数调用之后返回的协程对象
        - 协程队形 == 特殊的函数
        - 一个函数表示指定的一组操作，那么一个协程对象表示一组特定的操作
        - 如何创建一个协程？
            - 特殊函数的返回值
    - 任务对象
        - 就是一个高级的协程对象
        - 任务对象==协程对象==特殊函数
        - 任务对象==特殊函数
        - 任务对象表示一组特定的操作
        - 如何创建一个任务对象
            - task=asynio.ensure_future(c) #c:协程对象
    - 事件循环：EventLoop
        - 对象
        - 创建时间循环对象
            loop=asyncio.get_event_loop()
        - 作用：
            - 用来装载任务\协程对象
                - 可以将事件循环当做一个容器，容器中存放的是多个任务对象
            - 如果事件循环存放了多个任务对象且事件循环启动后，事件循环对象就可以异步的将每一个任务兑现对应的指定操作进行执行
        - 任务对象储存且启动事件循环队形
             - loop.run_util_complete(task)#将一个任务对象进行储存
    - 问题：
        - 事件循环启动后，任务对象对应的指定操作（特殊函数）已经被执行，但是如果这组指定的操作有返回值，如何获取返回值？
            - 基于任务对象的回调函数
            - 如何任务对象绑定回调函数
                task.add_done_callback(parse)
                parse就是回调函数
                    parse必须有一个参数，参数：该函数的绑定者（任务对象）
                    result()返回的就是特殊函数的返回值
        - 如何确定任务对象在时间循环中是否是基于异步执行的？
            -将多个任务对象注册到事件循环中进行测试
    - wait()方法的作用：
        - 表示挂起
        - asyncio.wait(tasks)将任务列表中的每一个任务对象进行挂起
        - 挂起：让当前的任务对象交出cpu的使用权
- 注意事项：
    在特殊函数内部不能出现不支持异步的模块，否则异步效果就会被中断
- 支持异步网络请求的模块[aiohttp]
    - 环境安装：pip install aiohttp
    - aiohttp的编码的使用
        - 大致的框架：
            async def get_request(url):
                async with aiohttp.ClientSession() as sess:
                    async with  sess.get(url=url) as response:
                        page_text= response.text()
                        return page_text
        - 补充细节：
            - 在每个with 前面＋async关键字
            - 在每个阻塞操作前+上一个await关键字
          完整的代码：
          async def get_request(url):
            async with aiohttp.ClientSession() as sess:
                async with await sess.get(url=url) as response:
                    page_text= await response.text()
                    return page_text
            - await关键字可以确保在异步执行操作的过程中可以保证阻塞操作执行完毕
- 实战说明：
    - 异步操作的条件：
        将待跑去的页面的url单独储存在一个list中
    - 步骤：
        - 用requests将待跑去页面的url获取
        - 将url放入list中，使用多任务异步协程爬取列表中的页面数据

In [6]:
import requests
import asyncio
import time

urls=[
    'http://127.0.0.1:5000/sxl',
    'http://127.0.0.1:5000/bobo',
    'http://127.0.0.1:5000/tom',
]

async def get_request(url):
    print('get_request被调用！！！！')
    page_text=requests.get(url).text
    return page_text

## 定义一个回调函数
##注意：回调函数的必须有一个参数：该函数的绑定者
def parse(task):
    #.result（）特殊函数的返回值的获取方法
    page_text=task.result()
    print('i am task callback 特殊函数的返回值为：',page_text)

if __name__=="__main__":
    ##创建协程对象
    c=get_request('http://127.0.0.1:5000/sxl')
    ##创建任务对象（基于已有的协程对象创建任务对象）
    task=asyncio.ensure_future(c)
    #给该任务对象绑定一个回调函数
    task.add_done_callback(parse)


    ##创建一个事件循序对象
    loop=asyncio.get_event_loop()
    #将task'函数储存到eventloop中且启动该对象
    loop.run_until_complete(task)

RuntimeError: This event loop is already running

In [13]:
## 多任务协程
import requests
import asyncio
import time
import nest_asyncio
nest_asyncio.apply()
import aiohttp
urls = [
    'http://127.0.0.1:5000/sxl',
    'http://127.0.0.1:5000/bobo',
    'http://127.0.0.1:5000/tom',
]


# async def get_request(url):
#     print('get_request被调用！！！！')
#     page_text = requests.get(url).text
#     return page_text

##特殊函数：不可以出现不支持异步模块的代码出现，不可以使用requests模块
async def get_request(url):
    #使用aiohttp进行网络请求
    async with aiohttp.ClientSession() as sess:#实例化一个请求对象叫做sess
        #sess.get(url，header，params,proxy)
        #sess.post(url，header，data,proxy)
        #proxy参数的用法和requests不一样，剩下的参数和requests的用法不一样
        #proxy="http"//ip:port"
        async with await sess.get(url=url) as response:#调用get发请求
            ##text()返回字符串形式的响应的数据
            ##reader()返回bytes类型的响应的数据
            page_text=await response.text()#获取源码的数据
            return page_text


## 定义一个回调函数
##注意：回调函数的必须有一个参数：该函数的绑定者
##多任务的异步爬虫中的数据解析或者持久化储存的操作序需要放在回调函数中
def parse(task):
    #.result（）特殊函数的返回值的获取方法
    page_text = task.result()
    print('callback function')
    print('i am task callback 特殊函数的返回值为：', len(page_text))


if __name__ == "__main__":
    tasks=list()
    start=time.time()
    for url in urls:
        ##创建协程对象
        c = get_request(url)
        ##创建任务对象（基于已有的协程对象创建任务对象）
        task = asyncio.ensure_future(c)
        #给该任务对象绑定一个回调函数
        task.add_done_callback(parse)
        tasks.append(task)

    ##创建一个事件循序对象
    loop = asyncio.get_event_loop()
    #将task'函数储存到eventloop中且启动该对象
    loop.run_until_complete(asyncio.wait(tasks))
    print('run time:',time.time()-start)

callback function
i am task callback 特殊函数的返回值为： 1271
callback function
i am task callback 特殊函数的返回值为： 1271
callback function
i am task callback 特殊函数的返回值为： 1271
run time: 2.0192654132843018
